In [4]:
import json

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model


In [2]:
from tensorflow.keras.models import load_model

In [3]:
from keras.layers import LSTM, Input,Embedding,Dense,GlobalMaxPooling1D,Flatten

In [4]:
from tensorflow import keras

In [5]:
with open('intents11 (2).json') as intents:
    data=json.load(intents)
    
    

In [6]:
data

{'intents': [{'tag': 'machine_learning',
   'patterns': ['What is Machine Learning?', 'Explain Machine Learning.'],
   'responses': ['Machine Learning (ML) is a field of artificial intelligence that focuses on developing algorithms and models that allow computer systems to learn and make predictions or decisions without being explicitly programmed. Key points about Machine Learning include:',
    '- ML algorithms learn from data and identify patterns to make predictions or take actions.',
    '- Supervised learning uses labeled data to train models, while unsupervised learning discovers patterns in unlabeled data.',
    '- Common ML algorithms include decision trees, support vector machines, and neural networks.',
    '- ML finds applications in various domains, such as image recognition, natural language processing, and recommender systems.',
    '- The performance of ML models is evaluated using metrics like accuracy, precision, recall, and F1 score.']},
  {'tag': 'deep_learning',
  

In [12]:
tags=[]
inputs=[]
responses={}
for intent in data['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['patterns']:
        inputs.append(lines)
        tags.append(intent['tag'])

In [21]:
df=pd.DataFrame({'Inputs':inputs,'tags':tags})

In [9]:
df

,Inputs,tags
0,What is Machine Learning?,machine_learning
1,Explain Machine Learning.,machine_learning
2,What is Deep Learning?,deep_learning
3,Explain Deep Learning.,deep_learning
4,What is Natural Language Processing?,nlp
...,...,...
169,Ask me a question,riddle
170,Riddle,riddle
171,how old are you,age
172,when were you made,age


In [10]:
df['tags'].value_counts

<bound method IndexOpsMixin.value_counts of 0      machine_learning
1      machine_learning
2         deep_learning
3         deep_learning
4                   nlp
             ...       
169              riddle
170              riddle
171                 age
172                 age
173                 age
Name: tags, Length: 174, dtype: object>

In [11]:
from transformers import AutoModel,BertTokenizer

In [9]:
def tokenize_data(tokenizer, texts):
  """Tokenizes a list of text strings using the given BERT tokenizer.

  Args:
    tokenizer: A BERT tokenizer.
    texts: A list of text strings.

  Returns:
    A list of tokenized sentences, where each sentence is a list of token IDs.
  """

  tokenized_texts = []
  for text in texts:
    # Fix the error by removing the `return_tensors='pt'` argument.
    tokens = tokenizer(text, padding=True, truncation=True, max_length=128)['input_ids']
    tokenized_texts.append(tokens)

  return tokenized_texts

In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')


In [174]:
from sklearn.model_selection import train_test_split

In [14]:
x =tokenize_data(tokenizer, df['Inputs'])

In [15]:
x

[[101, 2054, 2003, 3698, 4083, 1029, 102],
 [101, 4863, 3698, 4083, 1012, 102],
 [101, 2054, 2003, 2784, 4083, 1029, 102],
 [101, 4863, 2784, 4083, 1012, 102],
 [101, 2054, 2003, 3019, 2653, 6364, 1029, 102],
 [101, 4863, 3019, 2653, 6364, 1012, 102],
 [101, 2054, 2003, 3274, 4432, 1029, 102],
 [101, 4863, 3274, 4432, 1012, 102],
 [101, 2054, 2024, 2784, 7011, 3489, 5746, 2015, 1029, 102],
 [101, 4863, 2784, 7011, 3489, 5746, 2015, 1012, 102],
 [101, 2054, 2024, 2784, 7011, 3489, 6981, 1029, 102],
 [101, 4863, 2784, 7011, 3489, 6981, 1012, 102],
 [101, 2054, 2024, 2784, 7011, 3489, 6876, 1029, 102],
 [101, 4863, 2784, 7011, 3489, 6876, 1012, 102],
 [101, 2054, 2024, 2784, 7011, 3489, 4871, 1029, 102],
 [101, 4863, 2784, 7011, 3489, 4871, 1012, 102],
 [101, 2129, 2064, 2784, 7011, 3489, 5746, 2015, 2022, 11156, 1029, 102],
 [101, 25952, 2784, 7011, 3489, 5746, 2015, 102],
 [101, 2129, 2064, 2784, 7011, 3489, 6981, 2022, 11156, 1029, 102],
 [101, 25952, 2784, 7011, 3489, 6981, 102],
 [10

In [14]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
X=pad_sequences(x)

In [1]:
from transformers import BertModel

In [19]:
bert_model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
embedding_layer = Embedding(
    input_dim=bert_model.config.vocab_size,
    output_dim=bert_model.config.hidden_size,
    weights=[bert_model.embeddings.word_embeddings.weight.detach()],
    trainable=False)

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
le=LabelEncoder()

In [22]:
y=le.fit_transform(df['tags'])

In [24]:
y

array([35, 35, 13, 13, 39, 39,  9,  9, 14, 14, 20, 20, 21, 21, 19, 19, 15,
       15, 17, 17, 18, 18, 16, 16, 10, 10, 52, 52, 24, 24, 56, 56,  3,  3,
       37, 37, 44, 44, 36, 36, 26, 26, 11, 11,  5,  5,  6,  6,  6, 25, 25,
       25, 28, 28, 28, 28, 28, 28, 27, 27, 27, 27, 27, 27, 53, 53, 53, 53,
       49, 49, 49, 46, 46, 46, 46, 23, 23, 23, 23,  8,  8,  8,  8, 42, 42,
       42, 31, 31, 31, 50, 50, 50, 50, 50, 41, 41, 41, 41, 41, 34, 34, 34,
        0,  0, 12, 12, 12, 12, 12, 12, 12, 55, 55, 55, 55, 55, 30, 30, 30,
       30, 30, 43, 43, 43, 33, 33, 33,  1,  1, 22, 22, 22, 22, 22, 54, 54,
       54,  7,  7,  7,  7,  4,  4,  4,  4, 38, 38, 40, 40, 32, 32, 32, 47,
       47, 47, 47, 47, 29, 29, 29, 29, 29, 51, 48, 48, 48, 48, 48, 45, 45,
       45,  2,  2,  2])

In [26]:
outputlen=le.classes_.shape[0]
print(outputlen)

57


In [29]:
X.shape[1]

16

In [27]:
from tensorflow.keras.layers import Dropout

In [28]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, MultiHeadAttention




num_heads = 8
attention_dim = 64

# Assuming you have defined the input tensor, e.g., input_data
input_data = tf.keras.Input(shape=(X.shape[1]))

# Define the attention layer
attention = MultiHeadAttention(num_heads=num_heads, key_dim=attention_dim, value_dim=attention_dim)

# Connect the layers
x = embedding_layer(input_data)
attention_output = attention(query=x, key=x, value=x)  # Separate calls for query, key, and value
lstm_output = LSTM(18)(attention_output)

output = Dense(outputlen, activation='softmax')(lstm_output)


# Create the model
model = tf.keras.Model(inputs=input_data, outputs=output)


In [29]:
from tensorflow.keras.optimizers import Adam

In [30]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [31]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [32]:
model.fit(X, y, epochs=150,batch_size=32,)


Epoch 1/150
6/6 [==============================] - 3s 14ms/step - loss: 4.0562 - accuracy: 0.0115
Epoch 2/150
6/6 [==============================] - 0s 14ms/step - loss: 4.0377 - accuracy: 0.0115
Epoch 3/150
6/6 [==============================] - 0s 9ms/step - loss: 4.0142 - accuracy: 0.0230
Epoch 4/150
6/6 [==============================] - 0s 9ms/step - loss: 3.9955 - accuracy: 0.0287
Epoch 5/150
6/6 [==============================] - 0s 9ms/step - loss: 3.9852 - accuracy: 0.0517
Epoch 6/150
6/6 [==============================] - 0s 5ms/step - loss: 3.9643 - accuracy: 0.0287
Epoch 7/150
6/6 [==============================] - 0s 5ms/step - loss: 3.9290 - accuracy: 0.0345
Epoch 8/150
6/6 [==============================] - 0s 5ms/step - loss: 3.9282 - accuracy: 0.0517
Epoch 9/150
6/6 [==============================] - 0s 5ms/step - loss: 3.9370 - accuracy: 0.0402
Epoch 10/150
6/6 [==============================] - 0s 5ms/step - loss: 3.8689 - accuracy: 0.0460
Epoch 11/150
6/6 [=========

In [33]:
model.fit(X, y, epochs=50,batch_size=32,)


Epoch 1/50
6/6 [==============================] - 0s 10ms/step - loss: 1.2466 - accuracy: 0.7471
Epoch 2/50
6/6 [==============================] - 0s 10ms/step - loss: 1.2358 - accuracy: 0.7586
Epoch 3/50
6/6 [==============================] - 0s 10ms/step - loss: 1.2301 - accuracy: 0.7644
Epoch 4/50
6/6 [==============================] - 0s 7ms/step - loss: 1.2262 - accuracy: 0.7701
Epoch 5/50
6/6 [==============================] - 0s 5ms/step - loss: 1.2264 - accuracy: 0.7701
Epoch 6/50
6/6 [==============================] - 0s 5ms/step - loss: 1.1995 - accuracy: 0.7644
Epoch 7/50
6/6 [==============================] - 0s 5ms/step - loss: 1.2079 - accuracy: 0.7644
Epoch 8/50
6/6 [==============================] - 0s 5ms/step - loss: 1.1918 - accuracy: 0.7471
Epoch 9/50
6/6 [==============================] - 0s 5ms/step - loss: 1.1821 - accuracy: 0.7586
Epoch 10/50
6/6 [==============================] - 0s 5ms/step - loss: 1.1668 - accuracy: 0.7816
Epoch 11/50
6/6 [===================

In [35]:
model.fit(X, y, epochs=300,batch_size=32)


Epoch 1/300
6/6 [==============================] - 0s 31ms/step - loss: 0.9427 - accuracy: 0.7874
Epoch 2/300
6/6 [==============================] - 0s 11ms/step - loss: 0.9370 - accuracy: 0.8046
Epoch 3/300
6/6 [==============================] - 0s 5ms/step - loss: 0.9185 - accuracy: 0.8103
Epoch 4/300
6/6 [==============================] - 0s 5ms/step - loss: 0.9077 - accuracy: 0.8161
Epoch 5/300
6/6 [==============================] - 0s 5ms/step - loss: 0.9230 - accuracy: 0.8161
Epoch 6/300
6/6 [==============================] - 0s 5ms/step - loss: 0.9124 - accuracy: 0.8046
Epoch 7/300
6/6 [==============================] - 0s 5ms/step - loss: 0.9028 - accuracy: 0.8161
Epoch 8/300
6/6 [==============================] - 0s 5ms/step - loss: 0.8871 - accuracy: 0.8333
Epoch 9/300
6/6 [==============================] - 0s 5ms/step - loss: 0.8969 - accuracy: 0.8046
Epoch 10/300
6/6 [==============================] - 0s 5ms/step - loss: 0.8948 - accuracy: 0.8046
Epoch 11/300
6/6 [=========

6/6 [==============================] - 0s 5ms/step - loss: 0.3887 - accuracy: 0.9310
Epoch 168/300
6/6 [==============================] - 0s 5ms/step - loss: 0.3866 - accuracy: 0.9368
Epoch 169/300
6/6 [==============================] - 0s 5ms/step - loss: 0.3847 - accuracy: 0.9368
Epoch 170/300
6/6 [==============================] - 0s 5ms/step - loss: 0.3832 - accuracy: 0.9310
Epoch 171/300
6/6 [==============================] - 0s 5ms/step - loss: 0.3811 - accuracy: 0.9368
Epoch 172/300
6/6 [==============================] - 0s 5ms/step - loss: 0.3798 - accuracy: 0.9368
Epoch 173/300
6/6 [==============================] - 0s 5ms/step - loss: 0.3781 - accuracy: 0.9310
Epoch 174/300
6/6 [==============================] - 0s 5ms/step - loss: 0.3764 - accuracy: 0.9310
Epoch 175/300
6/6 [==============================] - 0s 5ms/step - loss: 0.3746 - accuracy: 0.9368
Epoch 176/300
6/6 [==============================] - 0s 5ms/step - loss: 0.3735 - accuracy: 0.9368
Epoch 177/300
6/6 [=====

In [52]:
model.fit(X, y, epochs=200
          ,batch_size=128)


Epoch 1/200
2/2 [==============================] - 0s 5ms/step - loss: 0.0876 - accuracy: 0.9828
Epoch 2/200
2/2 [==============================] - 0s 5ms/step - loss: 0.0875 - accuracy: 0.9828
Epoch 3/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0874 - accuracy: 0.9828
Epoch 4/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0874 - accuracy: 0.9828
Epoch 5/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0873 - accuracy: 0.9828
Epoch 6/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0872 - accuracy: 0.9828
Epoch 7/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0871 - accuracy: 0.9828
Epoch 8/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0870 - accuracy: 0.9828
Epoch 9/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0869 - accuracy: 0.9828
Epoch 10/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0868 - accuracy: 0.9828
Epoch 11/200
2/2 [===========

2/2 [==============================] - 0s 7ms/step - loss: 0.0758 - accuracy: 0.9828
Epoch 168/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0758 - accuracy: 0.9828
Epoch 169/200
2/2 [==============================] - 0s 5ms/step - loss: 0.0757 - accuracy: 0.9828
Epoch 170/200
2/2 [==============================] - 0s 7ms/step - loss: 0.0757 - accuracy: 0.9828
Epoch 171/200
2/2 [==============================] - 0s 5ms/step - loss: 0.0756 - accuracy: 0.9828
Epoch 172/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0755 - accuracy: 0.9828
Epoch 173/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0755 - accuracy: 0.9828
Epoch 174/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0754 - accuracy: 0.9828
Epoch 175/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0753 - accuracy: 0.9828
Epoch 176/200
2/2 [==============================] - 0s 5ms/step - loss: 0.0753 - accuracy: 0.9828
Epoch 177/200
2/2 [=====

In [41]:

model.save('chatbotdeepfake2.h5')

In [54]:
from tensorflow.keras.models import load_model

In [7]:
model=load_model('chatbotdeepfake2.h5')

In [25]:
import random
while True:
    text_p = []
    prediction_input = input('you: ')
    text_p.append(prediction_input)
    prediction_input = tokenize_data(tokenizer, text_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], 16)
    prediction = model.predict(prediction_input)
    output = prediction.argmax()
    print(output)
    response_tag = le.inverse_transform([output])[0]
    print(response_tag)

    for intent in data["intents"]:
        if intent['tag'] == response_tag:
            responses = intent['responses']
            print('chatbot:', random.choice(responses))
       



you: hello
1/1 [==============================] - 0s 12ms/step
28
greeting
chatbot: Hello! How can I assist you today?
you: how r you
1/1 [==============================] - 0s 12ms/step
55
whatsup
chatbot: All good..What about you?
you: who made u
1/1 [==============================] - 0s 12ms/step
43
programmer
chatbot: I was made by DeepGuard developers.
you: lol
1/1 [==============================] - 0s 14ms/step
30
haha
chatbot: Glad I could make you laugh !
you: idiot
1/1 [==============================] - 0s 11ms/step
33
insult
chatbot: Well that hurts :(
you: ty
1/1 [==============================] - 0s 13ms/step
48
suggest
chatbot: Please mail your suggestions to Pixel.Deepguard.suggestions@DeepGuard application Thank you for helping me improve!
you: cya bye
1/1 [==============================] - 0s 13ms/step
30
haha
chatbot: Glad I could make you laugh !
you: see you bye
1/1 [==============================] - 0s 14ms/step
27
goodbye
chatbot: Farewell! Feel free to return whene

KeyboardInterrupt: Interrupted by user